In [ ]:
from air_benchmark import AIRBench, Retriever, Reranker

class MyRetriever(Retriever):
    pass

class MyReranker(Reranker):
    pass


retriever = MyRetriever(search_top_k=1000)
reranker = MyReranker(rerank_top_k=100)

evaluation = AIRBench(
    benchmark_version="AIR-Bench_24.05",
    task_types=["long-doc"],    # remove this line if you want to evaluate on all task types
    domains=["arxiv"],          # remove this line if you want to evaluate on all domains
    languages=["en"],           # remove this line if you want to evaluate on all languages
    splits=["test"],            # remove this line if you want to evaluate on all splits
    cache_dir="<CACHE_DIR>"     # path to the cache directory (**NEED ~103GB FOR FULL 24.05 BENCHMARK**)
)

evaluation.run(
    retriever,
    reranker=reranker,          # remove this line if you don't have a reranker
    output_dir="<OUTPUT_DIR>"   # path to the output directory, default is "./search_results"
    overwrite=False             # set to True if you want to overwrite the existing results
)

In [1]:
from air_benchmark import AIRBench
from air_benchmark.tasks import LATEST_BENCHMARK_VERSION

evaluate_args = dict(
    benchmark_version = "AIR-Bench_24.05",
    search_results_save_dir = "./hf-transformers/search_results",
    cache_dir = "~/.cache",
    output_method = "markdown",
    output_path = "./24.05_eval_results.md",
    metrics = ["ndcg_at_10", "recall_at_10"],
)

AIRBench.evaluate_dev(**evaluate_args)

/mnt/disk1/JXH/01_apps/miniforge3/envs/finbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: './hf-transformers/search_results'